#Face Verification Example with DeepFace
This repository demonstrates face verification using DeepFace with considerations for different approaches and trade-offs.

In [ ]:
!pip install deepface opencv-python numpy

In [2]:
# Install unzip if not already installed
!apt-get install unzip

# Unzip the file
!unzip -q '/content/images.zip' -d '/content/extracted_images'


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


Extract Embeddings from images folder

In [4]:
from deepface import DeepFace
import os

# Path to your face database
db_path = "/content/extracted_images/images"
embeddings = []

# Extract embeddings for each image
for img_name in os.listdir(db_path):
    img_path = os.path.join(db_path, img_name)
    #change different embeddings extraction models for your requirement
    embedding = DeepFace.represent(img_path, model_name="VGG-Face")[0]["embedding"]
    embeddings.append(embedding)

# Save embeddings to a file (using numpy)
import numpy as np
np.save("my_embeddings.npy", embeddings)

25-02-04 12:50:08 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:03<00:00, 155MB/s] 


Test the verification with new data , adjust the thresholds.

In [17]:
# Load your stored embeddings
stored_embeddings = np.load("my_embeddings.npy")

# Path to the new test image
test_img_path = "/content/unknown1.jpg" #test with new elon musk images that not in images folder

# Extract embedding of the test image
test_embedding = DeepFace.represent(test_img_path, model_name="VGG-Face", detector_backend='opencv')[0]["embedding"] #change the models and detectors based on your requirement

# Compare with stored embeddings (using cosine similarity)
from numpy.linalg import norm

similarities = []
for emb in stored_embeddings:
    similarity = np.dot(emb, test_embedding) / (norm(emb) * norm(test_embedding))
    similarities.append(similarity)

# Check if any similarity is above a threshold (e.g., 0.7)
threshold = 0.5
is_verified = max(similarities) > threshold
print(f"Is the same person? {is_verified}")

Is the same person? True


**NOTE: Every time you change the models remove the my_embeddings.npy file then run the code to extract the embeddings with new model otherwise embeddings are mixed up and drops performance.**

**In application-level face verification, selecting the appropriate model depends on your specific requirements, such as on-device processing, cloud integration, or GPU support. It's important to balance model performance with inference time to meet your application's needs.**

**When you extract embeddings from a face and search through a database for matches, the process is known as face recognition. In our case, using a limited set of images (e.g., five) for matching is referred to as face verification.**

**There are numerous pretrained models available for face recognition tasks. Alternatively, you can train a model from scratch using convolutional neural networks (CNNs). For instance, OpenCV provides tools for face recognition, and you can find tutorials on implementing face recognition with OpenCV and Python. Additionally, cloud-based face recognition services offer user-friendly solutions but may involve ongoing costs and raise privacy concerns.**




**Model Comparison Table**

Recommended Use Case

Model	     __ Accuracy __ Speed	__ Input __  Size


VGG-Face    |High	Slow	 |224x224	  |High accuracy requirements

Facenet	  |Very High	    |Medium	  |160x160	Production systems

OpenFace	|Medium	Fast    |96x96	    |Mobile/Edge devices

DeepID	  |High	Fast	 |55x47	    |Low-resource environments

ArcFace	  |SOTA	Slow	 |112x112	  |Cutting-edge applications


#Detector Backends
* OpenCV: Fast but less accurate

* SSD: Balance of speed and accuracy

* MTCNN: Slower but handles difficult angles

* RetinaFace: Most accurate (especially for occluded faces) but slowest


**Threshold Guide**

Model _	Cosine Threshold _ Euclidean Threshold

VGG-Face	0.4	0.55

Facenet	0.3	0.75

OpenFace	0.1	0.9

ArcFace	0.4	0.5


In face recognition systems, similarity scores such as cosine similarity and Euclidean distance are commonly used to compare facial embeddings. However, these scores alone may not provide a complete picture of a model's performance due to variations in how different models extract and represent embeddings. Therefore, it's essential to employ comprehensive evaluation metrics to assess the effectiveness of face recognition models accurately.

**Evaluation Metrics:**

**Accuracy:** Measures the proportion of correct predictions (both true positives and true negatives) out of all predictions. While straightforward, accuracy can be misleading, especially with imbalanced datasets.

**Precision:** Indicates the proportion of true positive identifications among all positive identifications made by the model. High precision means fewer false positives.

**Recall (Sensitivity):** Reflects the proportion of true positive identifications among all actual positive instances. High recall indicates fewer false negatives.

**F1 Score:** The harmonic mean of precision and recall, providing a single metric that balances both concerns. It's particularly useful when dealing with imbalanced datasets.

**Receiver Operating Characteristic (ROC) Curve and Area Under the Curve (AUC):** The ROC curve plots the true positive rate against the false positive rate at various threshold settings, and the AUC summarizes the overall ability of the model to discriminate between positive and negative classes.

**Manual Testing:**

Beyond quantitative metrics, manual testing is crucial to ensure the model meets specific user requirements. This involves evaluating the model's performance on a diverse set of images, including variations in lighting, angles, expressions, and occlusions, to identify potential weaknesses and areas for improvement.

By combining similarity scores with these evaluation metrics and thorough manual testing, you can gain a more comprehensive understanding of a face recognition model's performance and its suitability for your specific application.

#Performance Considerations

**Embedding Extraction:**

Fastest: OpenFace (~300ms/face on CPU)

Slowest: ArcFace (~1.2s/face on CPU)

**Memory Usage:**

Lightest: OpenFace (4.3MB model)

Heaviest: VGG-Face (528MB model)

**Alternative Approaches**

Real-Time Verification:



In [ ]:
config = {
    'model_name': 'OpenFace',
    'detector_backend': 'ssd',
    'threshold': 0.2
}

High-Security Systems:

In [ ]:
config = {
    'model_name': 'ArcFace',
    'detector_backend': 'retinaface',
    'threshold': 0.5
}